In [1]:
import os

In [2]:
%pwd

'd:\\DS_Stats\\E2E_DS_Propjects\\ML_Project_Three\\research'

In [4]:
os.chdir('../')
%pwd

'd:\\DS_Stats\\E2E_DS_Propjects\\ML_Project_Three'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path

In [6]:
from src.chicken_disease_CNN.constants import *
from src.chicken_disease_CNN.utils.common import read_yaml,create_directories
from src.chicken_disease_CNN.entity.config_entity import DataIngestionConfig


class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
            ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
    
        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
from src.chicken_disease_CNN.logging import logger
from src.chicken_disease_CNN.utils.common import get_size
from src.chicken_disease_CNN.config.configuration import DataIngestionConfig
from pathlib import Path

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f'{filename} downloaded! with following info: \n{headers}')

        else:
            logger.info(f'File of name {filename} already exists with size {get_size(Path(self.config.local_data_file))}')


    def extract_zip_file(self):
        ''' 
        Arg:
            zip_file_path:str

        Extracts the zip file into data directory
        Function returns none 
        '''    

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
import sys
from src.chicken_disease_CNN.exception import CustomExeption

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
            
except Exception as e:
    raise CustomExeption(e,sys)